In [ ]:
#######################################################################################################################
# Project: Deep Virtual Rapport Agent (data preprocessing)
#
#     Jan Ondras (jo951030@gmail.com)
#     Institute for Creative Technologies, University of Southern California
#     April-October 2019
#
#######################################################################################################################
# Annotate OpenFace features from the ccdb dataset with ground-true head gestures.
#
#     Each frame will be annotated with ground-truth annotations of nod, shake, and tilt head gestures.
#
#     So far only for the basic set of the first 8 sessions.
#
#     Note: the script to annotate features using the developed Head Gesture Detector (and not the ground-truths) is 
#           deep-virtual-rapport-agent/head_gesture_detector/hgd_annotate_frames.ipynb and the corresponding annotated 
#           features are in dvra_datasets/ccdb/hgd_annotated_features
# 
#     Input features: dvra_datasets/ccdb/openface_features 
#     Input annotations: dvra_datasets/ccdb/original_data 
#     Output features: dvra_datasets/ccdb/annotated_features
#######################################################################################################################

In [1]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
###########################################################

# For each recording
#     Resample feature dataframe
#     Add first derivatives of selected features
#     Annotate frames
#     Save as new annotated dataframe, 0 => not a nod, 1 => nod

import os
import glob
import pandas as pd
import scipy.signal
from scipy import interpolate
import time
import pympi    # Import pympi to work with elan files

# Unified frame rate
FRAME_RATE = 30.

# Features whose first and second derivatives will be calculated
diff_selected_features = [
    ' pose_Tx', 
    ' pose_Ty', 
    ' pose_Tz', 

    ' pose_Rx', 
    ' pose_Ry', 
    ' pose_Rz',

    ' p_rx', 
    ' p_ry',
    ' p_rz'
    
    # add landmarks?
]

# For elan annotations
tier_name_to_head_gesture = {
    'Head Nodding': 'nod', 
    'Head Sideways Shake': 'shake', 
    'Head Tilt (left/right)': 'tilt'
}

input_features_dir = '/home/ICT2000/jondras/dvra_datasets/ccdb/openface_features'
input_annotations_dir = '/home/ICT2000/jondras/dvra_datasets/ccdb/original_data'
output_dir = '/home/ICT2000/jondras/dvra_datasets/ccdb/annotated_features'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
start_time = time.time()    
cnt = 0
# Consider only the basic ccdb dataset (first 8 sessions, i.e., 16 recordings)
for feature_file in sorted(glob.glob(input_features_dir + '/*_*_*_*.csv')):
    
    sessid = feature_file.split('/')[-1].split('.')[0]
    print(f'Processing SESSID {sessid}')
    
    org_df = pd.read_csv(feature_file)
    print(len(org_df))
#     print(len(org_df), org_df)
    
    # Resample feature dataframe to common frame rate, if needed
    csv_frame_rate = (org_df.shape[0] - 1) / np.sum(np.diff(org_df[' timestamp']))
    print(f'\tcsv frame rate: {csv_frame_rate}')
    if round(csv_frame_rate) == FRAME_RATE:
        print(f'\tNOT resampling')
        new_df = org_df.copy()
    else:
        print(f'\tREsampling from {csv_frame_rate} to {FRAME_RATE}')
        new_df = []
        timestamps_resampled = np.arange(0., org_df.iloc[-1][' timestamp'], step=1. / FRAME_RATE)
        for col_name in org_df.columns:            
            # Get interpolation function
            f = interpolate.interp1d(x=org_df[' timestamp'], y=org_df[col_name], kind='linear')
            new_df.append( f(timestamps_resampled) )
        new_df = pd.DataFrame(np.array(new_df).T, columns=org_df.columns) 
                
    # Add first and second derivatives of selected features
    diff_features = dict()
    for feature_name in diff_selected_features:
        diff_features['diff_' + feature_name] =  np.diff(new_df[feature_name], prepend=new_df[feature_name][0])
        diff_features['diff2_' + feature_name] = np.diff(diff_features['diff_' + feature_name], 
                                                         prepend=diff_features['diff_' + feature_name][0])    
    new_df = new_df.assign(**diff_features)
    
    # Annotate frames
    
    # Add annotation columns for none, nod, shake, and tilt class
    new_df = new_df.assign(none=np.zeros(len(new_df), dtype=int))
    new_df = new_df.assign(nod=np.zeros(len(new_df), dtype=int))
    new_df = new_df.assign(shake=np.zeros(len(new_df), dtype=int))
    new_df = new_df.assign(tilt=np.zeros(len(new_df), dtype=int))
    
    eaf_obj = pympi.Elan.Eaf(f'{input_annotations_dir}/{sessid[:-3]}/{sessid}/{sessid}.eaf')
    for tier_name in eaf_obj.get_tier_names():
        # Ignore annotations other than head nods, shakes, and tilts
        if tier_name in tier_name_to_head_gesture.keys():
            head_gesture = tier_name_to_head_gesture[tier_name]
            anns = eaf_obj.get_annotation_data_for_tier(tier_name)
            # Iterate over annotation intervals (in milliseconds)
            for ann in anns:
                interval_begin = ann[0] / 1000.
                interval_end   = ann[1] / 1000.
                new_df[head_gesture] = pd.np.where(
                    (new_df[' timestamp'] >= interval_begin) & (new_df[' timestamp'] <= interval_end), 1, 
                    new_df[head_gesture])
#                 print(interval_begin, interval_end, head_gesture)
#                 print(np.where(np.array(new_df.nod) == 1))
#                 print(np.where(np.array(new_df.shake) == 1))
#                 print(np.where(np.array(new_df.tilt) == 1))

    # Add column for None class
    new_df['none'] = np.where((new_df['nod'] == 0) & (new_df['shake'] == 0) & (new_df['tilt'] == 0), 1, new_df['none'])
    
    # Save as new annotated dataframe
#     print(new_df)
    new_df.to_csv(f'{output_dir}/{sessid}.csv', index=False)
    cnt += 1
    print(f'Time taken: {time.time() - start_time} s\n')        
#     break

print(f'\nGenerated {cnt} annotated feature files.')

Processing SESSID P1_P2_1402_C1
8821
	csv frame rate: 30.0
	NOT resampling
Time taken: 10.620553493499756 s

Processing SESSID P1_P2_1402_C2
8821
	csv frame rate: 30.0
	NOT resampling
Time taken: 21.017799615859985 s

Processing SESSID P1_P3_1502_C1
9805
	csv frame rate: 30.0
	NOT resampling
Time taken: 32.20632767677307 s

Processing SESSID P1_P3_1502_C2
8915
	csv frame rate: 30.000033654962593
	NOT resampling
Time taken: 42.67969584465027 s

Processing SESSID P3_P4_1502_C1
8901
	csv frame rate: 29.9999662921727
	NOT resampling
Time taken: 53.027095079422 s

Processing SESSID P3_P4_1502_C2
8901
	csv frame rate: 29.9999662921727
	NOT resampling
Time taken: 63.017903566360474 s

Processing SESSID P5_P2_1003_C1
8571
	csv frame rate: 29.99996499420654
	NOT resampling
Time taken: 73.06951975822449 s

Processing SESSID P5_P2_1003_C2
8571
	csv frame rate: 29.99996499420654
	NOT resampling
Time taken: 82.97037816047668 s

Processing SESSID P5_P3_2202_C1
8951
	csv frame rate: 30.00003351959052